# Membros do Grupo
* Daniele Montenegro da Silva Barros
* Rodrigo Dantas da Silva
* Thiago Bruschi Martins


# Pacote de dados


*   Abra o link do pacote de dados: https://tinyurl.com/bd10-auxilio
*   Selecione a opção "Adicionar Atalho ao Drive"


In [3]:
!pip install mrjob

     |████████████████████████████████| 439 kB 5.1 MB/s 


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls "/content/drive/My Drive/auxilio/"

 auxilio_0.1pct.csv.gz
 auxilio_10pct.csv.gz
 auxilio_1pct.csv.gz
 auxilio.csv
 auxilio.csv.gz
'Dicionário de Dados - Auxílio Emergencial - Portal da transparência.pdf'
 estimativa_dou_2021.csv


In [4]:
!zcat "/content/drive/My Drive/auxilio/auxilio.csv.gz" | head

"MÊS DISPONIBILIZAÇÃO";"UF";"CÓDIGO MUNICÍPIO IBGE";"NOME MUNICÍPIO";"NIS BENEFICIÁRIO";"CPF BENEFICIÁRIO";"NOME BENEFICIÁRIO";"NIS RESPONSÁVEL";"CPF RESPONSÁVEL";"NOME RESPONSÁVEL";"ENQUADRAMENTO";"PARCELA";"OBSERVAÇÃO";"VALOR BENEFÍCIO"
"202011";"";"";"";"00000000000";"***.643.870-**";"";"-2";"";"Não se aplica";"EXTRA CADUN";"7ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.108.816-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"6ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.095.215-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"6ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.108.816-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"7ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.885.226-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"5ª";"Não há";"600,00"
"202011";"";"";"";"00000000000";"***.885.226-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"3ª";"Não há";"600,00"
"202011";"";"";"";"00000000000";"***.885.226-**";"";"-2";"";"Nã

In [189]:
%%file auxilio.py
from mrjob.job import MRJob
from mrjob.protocol import TextProtocol
import re
import logging
import sys

class Auxilio(MRJob):

  OUTPUT_PROTOCOL = TextProtocol

  def configure_args(self):
    super(Auxilio, self).configure_args()
    self.add_passthru_arg('--popFile', default="/content/drive/My Drive/auxilio/estimativa_dou_2021.csv")

  def mapper(self, _, line):
    try :
      campos = line.replace('"','').lower().split(';') # Separa e remove as aspas dos campos
      codmun = str(campos[2])
      valor = str(campos[-1]).split(',')[0]
      
      # retorna o codigo do municipio e o valor pago
      yield codmun, int(valor)
    except :
      logging.warning("linha descartada: "+line)
      pass

  def reducer_init(self):
    logging.warning("criando dicionário de municipios")
    popFile = open(self.options.popFile, "r")
    self.popDict = {}
    for line in popFile :
      try :
        uf,coduf,codmun,nome,pop = line.split(',')
        self.popDict[coduf+codmun] = [nome, int(pop)]
      except:
        pass      
    popFile.close()

  def reducer(self, key, values):
    try :
      # Recupera o nome e a populacao da cidade
      nome_cidade, pop = self.popDict[str(key)]

      # Calcula o total de pagamento dividido pela populacao
      pagamento_per_capita = round(sum(values) / pop, 2)
    
      yield str(nome_cidade), str(pagamento_per_capita)

    except BaseException as err:
      logging.warning(sys.exc_info()[1])

if __name__ == '__main__':
    logging.basicConfig(filename="log.txt")
    Auxilio.run()

Overwriting auxilio.py


In [ ]:
!rm log.txt
!python auxilio.py "/content/drive/My Drive/auxilio/auxilio_0.1pct.csv.gz" > "/content/drive/My Drive/pagamento_per_capita_01.txt"

In [ ]:
!cat log.txt


In [191]:
# Executa o script e salva a saida no arquito txt
!python auxilio.py "/content/drive/My Drive/auxilio/auxilio.csv.gz"  > "/content/drive/My Drive/pagamento_per_capita.txt"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/auxilio.root.20211029.143736.929087
Running step 1 of 1...
job output is in /tmp/auxilio.root.20211029.143736.929087/output
Streaming final output from /tmp/auxilio.root.20211029.143736.929087/output...
Removing temp directory /tmp/auxilio.root.20211029.143736.929087...


In [ ]:
drive.flush_and_unmount()